Load deps

In [2]:
import numpy as np
import tensorflow as tf
import sys
import os

sys.path.insert(0, f"{os.getcwd()}/../../../../ML/preprocess/encoding/ascii")
from encoding import encode


Get data

In [3]:
x_train, y_train = encode()


In [4]:
print(x_train[100][:][:])
print(np.shape(x_train[0][:][:]))
print(y_train[100][:][:])
print(np.shape(y_train[0][:][:]))

[0 0 0 0 0 0 0 0 0 1 0 0 0 1 1 1 0 1 0 0 0 0 0 1 0 1 0 0 1 1 0 0 1 0 0 0 0
 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
(165,)
[0 0 0 1 0 0 0 0 0 0 0 0 0 0 0]
(15,)


Build model

In [40]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import (
    Input,
    LeakyReLU,
    Dense,
    Dropout,
)

DROPOUT = 0.9

model = Sequential()
model.add(Input(shape=(165)))
model.add(Dense(165, activation="gelu"))
model.add(Dropout(DROPOUT))
model.add(Dense(75, activation="gelu"))
model.add(Dropout(DROPOUT))
model.add(Dense(34, activation="gelu"))
model.add(Dropout(DROPOUT))
model.add(Dense(22, activation="gelu"))
model.add(Dropout(DROPOUT))
model.add(Dense(15, activation="softmax"))
model.compile(
    optimizer="Adam", loss="CategoricalCrossentropy", metrics=["CategoricalAccuracy"],
)

model.summary()


Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_37 (Dense)            (None, 165)               27390     
                                                                 
 dropout_14 (Dropout)        (None, 165)               0         
                                                                 
 dense_38 (Dense)            (None, 75)                12450     
                                                                 
 dropout_15 (Dropout)        (None, 75)                0         
                                                                 
 dense_39 (Dense)            (None, 34)                2584      
                                                                 
 dropout_16 (Dropout)        (None, 34)                0         
                                                                 
 dense_40 (Dense)            (None, 22)               

Split train and test

In [43]:
NB_SAMPLES = 100000

# Take rand portion of the data train samples
np.random.seed(2022)
rand_indices = np.random.permutation(len(x_train))
rand_train_indices = rand_indices[:NB_SAMPLES]
rand_test_indices = rand_indices[NB_SAMPLES : NB_SAMPLES + 300]
xs = x_train[rand_train_indices]
ys = y_train[rand_train_indices]
x_test = x_train[rand_test_indices]
y_test = y_train[rand_test_indices]

In [44]:
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping

BATCH_SIZE = 1000
EPOCHS = 1000


# Add early stopping and tensorboard callbacks
early_stopping_callback = EarlyStopping(monitor="loss", patience=100)
tensorboard_callback = TensorBoard(".logdir")
callbacks = [tensorboard_callback, early_stopping_callback]

model.fit(
    x=xs,
    y=ys,
    batch_size=BATCH_SIZE,
    validation_split=0.1,
    shuffle=True,
    epochs=EPOCHS,
    callbacks=[callbacks],
)

model.save("../../../../models/ff_on_natural_data_model.h5")


Epoch 1/1000
90/90 [==============================] - 2s 18ms/step - loss: 89.1537 - categorical_accuracy: 0.0736 - val_loss: 28.2726 - val_categorical_accuracy: 0.0655
Epoch 2/1000
90/90 [==============================] - 2s 17ms/step - loss: 81.2179 - categorical_accuracy: 0.0729 - val_loss: 42.3582 - val_categorical_accuracy: 0.0697
Epoch 3/1000
90/90 [==============================] - 1s 17ms/step - loss: 145.0331 - categorical_accuracy: 0.0716 - val_loss: 29.9476 - val_categorical_accuracy: 0.0862
Epoch 4/1000
90/90 [==============================] - 1s 16ms/step - loss: 97.8716 - categorical_accuracy: 0.0707 - val_loss: 90.8470 - val_categorical_accuracy: 0.0294
Epoch 5/1000
90/90 [==============================] - 2s 18ms/step - loss: 137.1957 - categorical_accuracy: 0.0709 - val_loss: 85.4259 - val_categorical_accuracy: 0.0212
Epoch 6/1000
90/90 [==============================] - 2s 18ms/step - loss: 98.1052 - categorical_accuracy: 0.0706 - val_loss: 74.9123 - val_categorical_a

In [45]:
print("Evaluate on test data")
results = model.evaluate(x_test, y_test, batch_size=30)
print("test loss, test acc:", results)


Evaluate on test data
10/10 [==============================] - 0s 2ms/step - loss: 0.2721 - categorical_accuracy: 0.0967
test loss, test acc: [0.2721417546272278, 0.09666666388511658]
